In [1]:
import boto3
import json
import pandas as pd
import numpy as np
from datetime import timedelta
from sqlalchemy import create_engine
# from timezonefinder import TimezoneFinder
from IPython.display import display
from tqdm import tqdm

### Connect to database

In [2]:
#personal tocken
dsn = dict(host='application-db-ro.prod.arable.cloud',
           user='hong_tang',
           password='9nzGhN0duGXy',
           port='5432',
           dbname='arable'
          )
sqlalchemy_dsn = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**dsn)

In [3]:
def connect_db(dsn: str) -> str:
    cnx = create_engine(dsn)

    return cnx

In [4]:
def read_dsd_raw(cnx, device, begin, end):
    schema = 'dsd_raw'

    query_template = """
select * from device_data.{schema} as dr
-- select count(*) from device_data.{schema} as dr
where dr.device = '{device}' and dr.time >= '{start}' and dr.time < '{end}';
"""

    sql_query = query_template.format(schema=schema, device=device, start=begin, end=end)

    df = pd.read_sql_query(sql_query, cnx)

    return df

def read_raw(cnx, device, begin, end):
    schema_raw = 'raw'
    query_template_raw = """
select r.time, r.device, r.location, r.rh 
from device_data.{schema} as r
where r.device = '{device}' and r.time >= '{start}' and r.time < '{end}';
"""

    sql_query = query_template_raw.format(schema=schema_raw, device=device, start=begin, end=end)

    df = pd.read_sql_query(sql_query, cnx)

    return df

### Starting the engine

In [5]:
pg_conn = connect_db(sqlalchemy_dsn)

In [6]:
device_list=[ 'C005252', 'C004146', 'C004146', 'C004231', 'C004301', 'C005182', 'C007852']
#        'C007978', 'C007880', 'C003405', 'C003445', 'C003245', 'C004991',
#        'C004183', 'C006338', 'C004894', 'C003655', 'C005042', 'C004299',
#        'C005043', 'C004149', 'C004223', 'C005397', 'C004196', 'C006158',
#        'C006171', 'C007399', 'C006126', 'C006163', 'C004988', 'C004830',
#        'C003326', 'C005285', 'C003065', 'C003166', 'C003186', 'C003324',
#        'C003359', 'C003427', 'C003496', 'C003497', 'C004174', 'C003276',
#        'C005312', 'C005339', 'C005234', 'C004976', 'C003188', 'C003240',
#        'C006164', 'C003251', 'C006123', 'C004200', 'C005288', 'C003017',
#        'C004854', 'C006826', 'C003279', 'C003398', 'C005348', 'C003241',
#        'C003388', 'C003791', 'C005484', 'C003231', 'C006160', 'C003015',
#        'C005429', 'C005386'

In [7]:
def read_hourly(cnx, device, begin, end):
    schema_raw = 'hourly'
    query_template_raw = """
select h.time, h.device,  h.precip 
from device_data.{schema} as h
where h.device = '{device}' and h.time >= '{start}' and h.time < '{end}';
"""

    sql_query = query_template_raw.format(schema=schema_raw, device=device, start=begin, end=end)
#     print(sql_query)
    df = pd.read_sql_query(sql_query, cnx)

    return df
#, h.precip_classifier_version, h.precip_version #h.location,

In [8]:
len(df_final.device.unique()), len(device_list)

NameError: name 'df_final' is not defined

In [9]:
%time
series = []
df_final = pd.DataFrame()

start='2020-01-01'
end='2022-05-24'
for device in device_list:
    print(device, start, end)
    
    df_hourly = read_hourly(pg_conn, device, start, end)
    print(df_hourly.shape)
#     series.append(df_hourly)
    if not df_hourly.empty:
        
        df_final=pd.concat([df_final, df_hourly])

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs
C005252 2020-01-01 2022-05-24
(12563, 3)
C004146 2020-01-01 2022-05-24
(4881, 3)
C004146 2020-01-01 2022-05-24
(4881, 3)
C004231 2020-01-01 2022-05-24
(8105, 3)
C004301 2020-01-01 2022-05-24
(8263, 3)
C005182 2020-01-01 2022-05-24
(4564, 3)
C007852 2020-01-01 2022-05-24
(7951, 3)


In [10]:
df_final.tail(100)

,time,device,precip
7851,2021-08-29 08:00:00+00:00,C007852,0.37
7852,2022-04-02 16:00:00+00:00,C007852,0.23
7853,2021-08-08 21:00:00+00:00,C007852,0.00
7854,2021-08-08 23:00:00+00:00,C007852,0.00
7855,2021-12-09 22:00:00+00:00,C007852,0.00
...,...,...,...
7946,2022-04-14 16:00:00+00:00,C007852,0.00
7947,2021-08-31 05:00:00+00:00,C007852,0.01
7948,2021-08-31 16:00:00+00:00,C007852,0.00
7949,2021-08-31 15:00:00+00:00,C007852,0.00
